In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit


def colourbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    ax = mappable.axes
    figle = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    return figle.colorbar(mappable, cax=cax, format='%g')


def f(x, m, b):
    y = m * x + b
    return y


Epoch = 5
row, col = 4, 4

AC_Data = np.load('/home/broughtonco/documents/nrc/data/IC348/Datafiles/IC348_AC.npy')
PSD_Data = np.load('/home/broughtonco/documents/nrc/data/IC348/Datafiles/IC348_PSD.npy')
radius = np.load('/home/broughtonco/documents/nrc/data/IC348/Datafiles/IC348_radius.npy')
MetaData = np.loadtxt('/home/broughtonco/documents/nrc/data/IC348/IC348_850_EA3_cal_metadata.txt', dtype='str')
BaseDate = MetaData[0][2][1:]
Date = MetaData[Epoch - 1][2][1:]

for dist in [5, 10, 15, 20]:
    num = len(radius[radius <= dist])

    # Avg_AC = np.sum(AC_Data, axis=0)/len(AC_Data)
    # Avg_PSD = np.sum(PSD_Data, axis=0)/len(PSD_Data)

    DivACData, DivPSDData = [], []
    fit_ac, cov_ac = [], []
    fit_psd, cov_psd = [], []

    for i in range(0, len(AC_Data)):
        DivAC = AC_Data[i] / AC_Data[0]
        DivACData.append(DivAC)
        optimal_fit_AC, cov_mat_AC = curve_fit(f, radius[1:num], DivAC[1:num])
        fit_ac.append(optimal_fit_AC)
        cov_ac.append(cov_mat_AC)

        DivPSD = PSD_Data[i] / PSD_Data[0]
        DivPSDData.append(DivPSD)
        optimal_fit_PSD, cov_mat_PSD = curve_fit(f, radius[1:num], DivPSD[1:num])
        fit_psd.append(optimal_fit_PSD)
        cov_psd.append(cov_mat_PSD)

    # Plotting follows

    AC_text = 'y = {:g} \u00B1 {:g} x + {:g} \u00B1 {:g}'.format(fit_ac[Epoch][0],
                                                                 cov_ac[Epoch][0, 0],
                                                                 fit_ac[Epoch][1],
                                                                 cov_ac[Epoch][1, 1]
                                                                 )
    PSD_text = 'y = {:g} \u00B1 {:g} x + {:g} \u00B1 {:g}'.format(fit_psd[Epoch][0],
                                                                  cov_psd[Epoch][0, 0],
                                                                  fit_psd[Epoch][1],
                                                                  cov_psd[Epoch][1, 1]
                                                                  )
    fig = plt.figure(figsize=(10, 10))
    grid = plt.GridSpec(nrows=row, ncols=col, hspace=0.5, wspace=0.2)
    fig.text(0.5, 13 / (row*col), AC_text, horizontalalignment='center', verticalalignment='center')
    fig.text(0.5, 6 / (row*col), PSD_text, horizontalalignment='center', verticalalignment='center')

    # AC = fig.add_subplot(grid[0:2, :])
    AC_DIV = fig.add_subplot(grid[0:2, :])
    PSD_DIV = fig.add_subplot(grid[2:4, :])

    # AC.set_title('Auto-correlation')
    # AC.scatter(radius[1:num], AC_Data[Epoch - 1][1:num].real, s=1, marker=',', lw=0, color='red')
    # AC.set_yscale('log')
    # AC.set_ylim(bottom=10**-4, top=10**4)
    # AC.set_xticks(np.arange(0, dist+1, step=1))

    AC_DIV.set_title('AC Div Map')
    AC_DIV.scatter(radius[1:num], DivACData[Epoch - 1][1:num].real, s=1, marker=',', lw=0, color='red')
    AC_DIV.plot(radius[1:num], f(radius[1:num], fit_ac[Epoch - 1][0], fit_ac[Epoch - 1][1]),
                linestyle='--',
                color='green',
                linewidth=0.5)
    AC_DIV.set_xticks(np.arange(0, dist+1, step=1))
    AC_DIV.set_xlabel('radius from centre')
    AC_DIV.set_ylabel('Similarity in AC')

    # AC_DIV.set_yscale('log')
    # AC_DIV.set_ylim(bottom=10**-4, top=10**4)

    PSD_DIV.set_title('PSD Div Map')
    PSD_DIV.scatter(radius[1:num], DivPSDData[Epoch - 1][1:num].real, s=1, marker=',', lw=0, color='red')
    PSD_DIV.plot(radius[1:num], f(radius[1:num], fit_psd[Epoch - 1][0], fit_psd[Epoch - 1][1]),
                 linestyle='--',
                 color='green',
                 linewidth=0.5)
    PSD_DIV.set_xticks(np.arange(0, dist+1, step=1))
    PSD_DIV.set_xlabel('radius from centre')
    PSD_DIV.set_ylabel('Similarity in PSD')
    # PSD_DIV.set_yscale('log')
    # PSD_DIV.set_ylim(bottom=10**0, top=10**6)
    plt.suptitle('Epoch {} and Epoch {}'.format(Date, BaseDate))
    fig.savefig('/home/broughtonco/documents/nrc/data/IC348/IC348_plots/linearFit_radius_{}.png'.format(dist))
    # plt.show()

